# Probe + Brick Animation

Simple animation of the NetFT probe touching foam bricks.

This notebook visualizes the touch-hold-retreat motion without robot control complexity.

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path for ur5e_description
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePose,
    Quaternion,
    Rgba,
    RigidTransform,
    Role,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
)

from manipulation import ConfigureParser, running_as_notebook

In [ ]:
# Start Meshcat visualizer
meshcat = StartMeshcat()

# Define TCP Keyframes

Keyframes for touch-hold-retreat motion on two bricks:
- Approach brick A from above
- Touch and hold for 1 second
- Retreat
- Move to brick B
- Touch and hold for 1 second
- Retreat to initial position

In [ ]:
def MakeTCPFrames(X_TCP_initial, X_BrickA, X_BrickB):
    """
    Creates keyframe poses for touch-hold-retreat motion on two bricks.
    """
    X_TCP = {"initial": X_TCP_initial}
    
    # TCP orientation: Z-axis pointing down (toward brick)
    R_touch = RotationMatrix.MakeXRotation(np.pi)
    
    # Geometry constants
    # Foam brick base_link is at BOTTOM, so top surface is at Z = base_z + brick_height
    brick_height = 0.05
    approach_height = 0.08  # 8cm above brick top
    contact_offset = 0.0  # Just touch the surface (no penetration)
    
    # Brick A contact frames
    p_BrickA = X_BrickA.translation()
    top_A = p_BrickA[2] + brick_height  # base_link at bottom, top = base + height
    
    X_TCP["pretouch_A"] = RigidTransform(
        R_touch, 
        [p_BrickA[0], p_BrickA[1], top_A + approach_height]
    )
    X_TCP["touch_A"] = RigidTransform(
        R_touch,
        [p_BrickA[0], p_BrickA[1], top_A - contact_offset]
    )
    X_TCP["posttouch_A"] = X_TCP["pretouch_A"]
    
    # Clearance pose
    p_mid = (p_BrickA + X_BrickB.translation()) / 2
    X_TCP["clearance"] = RigidTransform(
        R_touch,
        [p_mid[0], p_mid[1], top_A + 0.15]
    )
    
    # Brick B contact frames
    p_BrickB = X_BrickB.translation()
    top_B = p_BrickB[2] + brick_height
    
    X_TCP["pretouch_B"] = RigidTransform(
        R_touch,
        [p_BrickB[0], p_BrickB[1], top_B + approach_height]
    )
    X_TCP["touch_B"] = RigidTransform(
        R_touch,
        [p_BrickB[0], p_BrickB[1], top_B - contact_offset]
    )
    X_TCP["posttouch_B"] = X_TCP["pretouch_B"]
    
    # Final pose
    X_TCP["final"] = X_TCP_initial
    
    # Timing
    speed_factor = 5.0  # seconds per meter
    times = {"initial": 0.0}
    
    dist = np.linalg.norm(X_TCP["pretouch_A"].translation() - X_TCP["initial"].translation())
    times["pretouch_A"] = times["initial"] + speed_factor * dist
    
    times["touch_A_start"] = times["pretouch_A"] + 1.0
    times["touch_A_end"] = times["touch_A_start"] + 1.0
    X_TCP["touch_A_start"] = X_TCP["touch_A"]
    X_TCP["touch_A_end"] = X_TCP["touch_A"]
    
    times["posttouch_A"] = times["touch_A_end"] + 1.0
    
    dist = np.linalg.norm(X_TCP["clearance"].translation() - X_TCP["posttouch_A"].translation())
    times["clearance"] = times["posttouch_A"] + speed_factor * dist
    
    dist = np.linalg.norm(X_TCP["pretouch_B"].translation() - X_TCP["clearance"].translation())
    times["pretouch_B"] = times["clearance"] + speed_factor * dist
    
    times["touch_B_start"] = times["pretouch_B"] + 1.0
    times["touch_B_end"] = times["touch_B_start"] + 1.0
    X_TCP["touch_B_start"] = X_TCP["touch_B"]
    X_TCP["touch_B_end"] = X_TCP["touch_B"]
    
    times["posttouch_B"] = times["touch_B_end"] + 1.0
    
    dist = np.linalg.norm(X_TCP["final"].translation() - X_TCP["posttouch_B"].translation())
    times["final"] = times["posttouch_B"] + speed_factor * dist
    
    return X_TCP, times


def MakeTCPPoseTrajectory(X_TCP, times):
    """Constructs PiecewisePose trajectory from keyframes."""
    keyframe_order = [
        "initial",
        "pretouch_A",
        "touch_A_start",
        "touch_A_end",
        "posttouch_A",
        "clearance",
        "pretouch_B",
        "touch_B_start",
        "touch_B_end",
        "posttouch_B",
        "final",
    ]
    
    sample_times = [times[name] for name in keyframe_order]
    poses = [X_TCP[name] for name in keyframe_order]
    
    return PiecewisePose.MakeLinear(sample_times, poses)

# Define Brick Positions and Initial Probe Pose

In [ ]:
# Brick positions (base_link frame is at brick BOTTOM, so z=0 means sitting on ground)
X_BrickA = RigidTransform([0.4, -0.2, 0.0])
X_BrickB = RigidTransform([0.4, 0.2, 0.0])

# Initial probe pose (above and between bricks)
X_TCP_initial = RigidTransform(
    RotationMatrix.MakeXRotation(np.pi),  # Z pointing down
    [0.4, 0.0, 0.25]  # 25cm above ground
)

# Generate keyframes and trajectory
X_TCP, times = MakeTCPFrames(X_TCP_initial, X_BrickA, X_BrickB)
print(f"Total animation duration: {times['final']:.2f} seconds")

# Create trajectory
traj_X_TCP = MakeTCPPoseTrajectory(X_TCP, times)

# Trajectory-to-Position System

This system samples the trajectory and sets the probe's free-body pose.

In [ ]:
class ProbeTrajectoryToPosition(LeafSystem):
    """Converts TCP trajectory to plant positions for visualization."""

    def __init__(self, plant, traj_X_TCP):
        LeafSystem.__init__(self)
        self.plant = plant
        self.probe_body = plant.GetBodyByName("netft_link")
        self.traj_X_TCP = traj_X_TCP
        self.plant_context = plant.CreateDefaultContext()

        # TCP offset from netft_link (from URDF: 0.05206 + 0.185 = 0.23706m)
        self.tcp_offset = RigidTransform([0, 0, 0.23706])

        self.DeclareVectorOutputPort(
            "position", plant.num_positions(), self.CalcPositionOutput
        )

    def CalcPositionOutput(self, context, output):
        t = context.get_time()

        # Get desired TCP pose from trajectory (use GetPose, not value)
        X_W_TCP = self.traj_X_TCP.GetPose(t)

        # Compute netft_link pose (TCP is offset from netft_link)
        X_W_netft = X_W_TCP.multiply(self.tcp_offset.inverse())

        # Set the free body pose
        self.plant.SetFreeBodyPose(self.plant_context, self.probe_body, X_W_netft)
        output.SetFromVector(self.plant.GetPositions(self.plant_context))

# Build and Run Animation

In [ ]:
def animate_probe_and_bricks(traj_X_TCP, X_BrickA, X_BrickB):
    """Build diagram and animate probe touching bricks."""
    builder = DiagramBuilder()
    
    # Create scene graph (no physics, just visualization)
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    
    parser = Parser(plant, scene_graph)
    ConfigureParser(parser)
    parser.package_map().Add('ur5e_description', str(Path.cwd().parent / 'ur5e_description'))
    parser.SetAutoRenaming(True)
    
    # Load probe as free-floating body
    probe = parser.AddModelsFromUrl(
        "package://ur5e_description/netft_probe.urdf"
    )[0]
    
    # Load and weld bricks using base_link frame (origin at brick BOTTOM)
    brick_a = parser.AddModelsFromUrl(
        "package://manipulation/hydro/061_foam_brick.sdf"
    )[0]
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("base_link", brick_a),
        X_BrickA
    )
    
    brick_b = parser.AddModelsFromUrl(
        "package://manipulation/hydro/061_foam_brick.sdf"
    )[0]
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("base_link", brick_b),
        X_BrickB
    )
    
    plant.Finalize()
    
    # Connect trajectory system to scene graph
    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(
        to_pose.get_output_port(),
        scene_graph.get_source_pose_port(plant.get_source_id())
    )
    
    # Trajectory to position system
    traj_to_pos = builder.AddSystem(ProbeTrajectoryToPosition(plant, traj_X_TCP))
    builder.Connect(traj_to_pos.get_output_port(), to_pose.get_input_port())
    
    # Add visualizers for both illustration and proximity roles
    meshcat.Delete()
    
    # Illustration (visual geometry)
    MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kIllustration, prefix="illustration")
    )
    
    # Proximity (collision geometry)
    MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kProximity, prefix="proximity")
    )
    
    # Draw trajectory path
    traj_p = traj_X_TCP.get_position_trajectory()
    p_traj = traj_p.vector_values(traj_p.get_segment_times())
    meshcat.SetLine("tcp_path", p_traj, 2.0, rgba=Rgba(1, 0.65, 0))
    
    diagram = builder.Build()
    
    # Run simulation
    simulator = Simulator(diagram)
    meshcat.StartRecording(set_visualizations_while_recording=False)
    simulator.AdvanceTo(traj_X_TCP.end_time() if running_as_notebook else 0.1)
    meshcat.PublishRecording()
    
    return diagram


# Run the animation
print("Animating probe and bricks...")
print(f"Watch in Meshcat: {meshcat.web_url()}")
diagram = animate_probe_and_bricks(traj_X_TCP, X_BrickA, X_BrickB)
print("Animation complete! Use the scrubber in Meshcat to replay.")